# Training our Fruit Classifer

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)

In [ ]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

num_classes = 131
img_rows, img_cols = 64, 64
batch_size = 128

from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training/'
validation_data_dir = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
#Training Data    
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
#Validation Data
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)


### Let's define our model

In [ ]:
#model = Sequential()

# Padding = 'same'  results in padding the input such that
# the output has the same length as the original input
#model.add(Conv2D(256, (3, 3), padding='same',input_shape= (img_rows, img_cols, 3)))
#model.add(Activation('relu'))
#model.add(Conv2D(128, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

#model.add(Conv2D(64, (3, 3), padding='same'))
#model.add(Activation('relu'))
#model.add(Conv2D(64, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

#model.add(Flatten())
#model.add(Dense(128))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
#model.add(Dense(num_classes))
#model.add(Activation('softmax'))

# initiate RMSprop optimizer and configure some parameters
#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
#print(model.summary())


model=Sequential()

#Convolution blocks
model.add(Conv2D(128, kernel_size = (3,3), padding='same',input_shape=(img_rows,img_cols,3),activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 

model.add(Conv2D(64, kernel_size = (3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 

#model.add(Conv2D(256, kernel_size = (3,3), padding='same',activation='relu'))
#model.add(MaxPooling2D(pool_size=2)) 

#Classification layers
model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))



model.add(Dense(num_classes,activation='softmax'))


model.summary()

In [ ]:
# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'RMSprop',
              metrics = ['accuracy'])

nb_train_samples = 67692
nb_validation_samples = 22688
epochs = 50

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save('fruit_classifier1.h5')

import pickle
pickle_out = open('fruit_classifier2.pkl','wb')
pickle.dump(history.history,pickle_out)
pickle_out.close()

pickle_in = open('fruit_classifier2.pkl','rb')
history=pickle.load(pickle_in)



In [ ]:
import pickle
pickle_in = open('fruit_classifier2.pkl','rb')
history=pickle.load(pickle_in)


### Displaying our Confusion Matrix

In [ ]:
class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())

from sklearn.metrics import classification_report, confusion_matrix

#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

print(validation_generator.classes)

In [ ]:
print(classes)

### Testing our fruit classifier

In [ ]:
from keras.models import load_model
model = load_model('fruit_classifier1.h5')


def print_result(result):
    i,j = np.unravel_index(result.argmax(), result.shape)
    print(f'Prediction: {classes[j]} | Probability: {result[i,j]} | {i},{j}')   

def predict_image_model(image_path : str) -> np.ndarray:
    img_size = (img_rows, img_cols)
    # load imamge into a 4D Tensor, convert it to a numpy array and expand to 4 dim
    img1 = image.load_img(image_path, target_size = img_size)
    image_tensor = image.img_to_array(img1)
    #print(image_tensor.shape)
    image_tensor = image_tensor/255
    image_tensor = np.expand_dims(image_tensor, axis=0)
    #print(img.shape)
    result = model.predict(image_tensor)
    return result

from keras.preprocessing import image
test_img_path_1  = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/Cherry Wax Red/191_100.jpg'
test_img_path_2 = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/Watermelon/r_95_100.jpg'
test_img_path_3  = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/Tomato not Ripened/r_297_100.jpg'
test_img_path_4 = '/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test/apple_crimson_snow_1/r1_59.jpg'
#result
result = predict_image_model(test_img_path_1)#Cherry Wax Red
print_result(result)
result = predict_image_model(test_img_path_2)#Watermelon
print_result(result)
result = predict_image_model(test_img_path_3)#Tomato not Ripened
print_result(result)
result = predict_image_model(test_img_path_4)#apple_crimson_snow_1
print_result(result)



 



In [ ]:
data_history = history
data_history['epochs']= [i for i in range(1,epochs+1)]
data_history

In [ ]:
from matplotlib import pyplot as plt
_epochs = data_history['epochs']
_loss = data_history['loss']
_accuracy = data_history['accuracy']
_val_accuracy = data_history['val_accuracy']
_val_loss = data_history['val_loss']

plt.plot(_epochs,_val_accuracy)


In [ ]:
plt.plot(_epochs,_accuracy)